<a href="https://colab.research.google.com/github/pythonkvs/seminars/blob/main/%D0%A1%D0%B5%D0%BC%D0%B8%D0%BD%D0%B0%D1%80_%D0%BA%D0%BB%D0%B0%D1%81%D1%81%D1%8B_23_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Синтаксис объявления классов

In [1]:
class TimeInterval:
    pass

In [2]:
interval = TimeInterval()

In [3]:
type(interval)

__main__.TimeInterval

In [4]:
type(TimeInterval)

type

In [5]:
TimeInterval is type(interval)

True

In [6]:
isinstance(interval, TimeInterval)

True

# Конструирование

In [7]:
class TimeInterval:
    def __init__(self, begin, end):
        self.begin = begin
        self.end = end

In [8]:
from datetime import datetime
interval = TimeInterval(
    datetime(year=2018, month=1, day=1),
    datetime(year=2019, month=1, day=2),
)

In [9]:
interval.begin

datetime.datetime(2018, 1, 1, 0, 0)

# Атрибуты

In [10]:
interval.xxx = 42
interval.xxx

42

In [11]:
interval.not_found

AttributeError: ignored

# Методы

In [12]:
class TimeInterval:
    def __init__(self, begin, end):
        self.begin = begin
        self.end = end
        
    def get_length(self):
        return self.end - self.begin

In [13]:
interval = TimeInterval(datetime(year=2016, month=1, day=1), datetime.now())
interval = TimeInterval(
    datetime(year=2018, month=1, day=1),
    datetime(year=2018, month=1, day=2),
)
interval.get_length().total_seconds()

86400.0

In [14]:
interval.unknown_method()

AttributeError: ignored

### `Связанные методы`

Связанные методы экземпляра - пара self + функция.

Попытка обращения к функциональному атрибуту класса через имя экземпляра возвращает объект связанного метода. Интерпретатор автоматически упаковывает экземпляр с функцией в объект связанного метода, поэтому вам не требуется передавать экземпляр в вызов такого метода.

In [15]:
class Spam:
    def doit(self, message):
        print(message)

object1 = Spam()
object1.doit('hello world')

hello world


Создается объект связанного метода object1.doit, в который упакованы вместе экземпляр object1 и метод Spam.doit

Можно присвоить этот объект переменной и использовать переменную для вызова как простую функцию:

In [16]:
x = object1.doit    # Объект связанного метода: экземпляр + функция
print(repr(x))

x('hello world')    # То же, что и object1.doit('...')

<bound method Spam.doit of <__main__.Spam object at 0x7f7086a0ffd0>>
hello world


### `Несвязанные методы (в прежнем виде нет, есть function)`

https://riptutorial.com/python/example/1401/bound--unbound--and-static-methods

Вызовем метод через имя класса:

In [17]:
object1 = Spam()
t = Spam.doit # раньше - объект несвязанного метода <unbound method Spam.doit> (in Python 2.x), 
              # сейчас просто function object <function __main__.Spam.doit>
print(repr(t))

t(object1, 'howdy') # Передать экземпляр в первый аргумент

<function Spam.doit at 0x7f7086a20b90>
howdy


# `__dict__`


- Для большого числа типов в Python определена переменная-словарь `__dict__`
- Она содержит атрибуты, специфичные для данного объекта (не его класса и не его родителей)
- Множество элементов `__dict__` является подмножеством элементов, возвращаемых функцией `dir()`

In [18]:
class A: pass

print(set(A.__dict__.keys()).issubset(set(dir(A))))

print(set(dir(A)))
print(set(A.__dict__.keys()))


[].__dict__

True
{'__hash__', '__eq__', '__subclasshook__', '__init_subclass__', '__weakref__', '__class__', '__sizeof__', '__doc__', '__dir__', '__le__', '__ge__', '__gt__', '__init__', '__str__', '__setattr__', '__delattr__', '__ne__', '__module__', '__lt__', '__format__', '__getattribute__', '__dict__', '__repr__', '__reduce_ex__', '__new__', '__reduce__'}
{'__dict__', '__module__', '__weakref__', '__doc__'}


AttributeError: ignored

# `__slots__`

In [19]:
class TimeInterval:
    __slots__ = ['_begin', '_end']
    
    def __init__(self, begin, end):
        self._begin = begin
        self._end = end
        
    def get_length(self):
        return self._end - self._begin

In [20]:
from datetime import datetime
interval = TimeInterval(datetime(2018, 1, 1), datetime(2018, 1, 2))
interval.xxx = 123

AttributeError: ignored

# Методы экземпляра класса, методы класса, статические методы

In [21]:
class ToyClass:
    def instancemethod(self):
        return 'instance method called', self
    
    @classmethod
    def classmethod(cls):
        return 'class method called', cls
    @staticmethod
    def staticmethod():
        return 'static method called'

### `Методы экземпляра класса`

Это наиболее часто используемый вид методов. Методы экземпляра класса принимают объект класса как первый аргумент, который принято называть self и который указывает на сам экземпляр. Количество параметров метода не ограничено.
Используя параметр self , мы можем менять состояние объекта и обращаться к другим его методам и параметрам. К тому же, используя атрибут `self.__class__` , мы получаем доступ к атрибутам класса и возможности менять состояние самого класса. То есть методы экземпляров класса позволяют менять как состояние определённого объекта, так и класса.

In [22]:
"welcome".upper()   # <- вызывается на строковых данных

'WELCOME'

### `Методы класса`

Методы класса привязаны к самому классу, а не его экземпляру. Они могут менять состояние класса, что отразится на всех объектах этого класса, но не могут менять конкретный объект.

In [23]:
dict.fromkeys('AEIOU')  # <- вызывается при помощи класса dict

{'A': None, 'E': None, 'I': None, 'O': None, 'U': None}

### `Статические методы`

Статические методы декларируются при помощи декоратора staticmethod. Им не нужен определенный первый аргумент (ни self, ни cls).
Их можно воспринимать как методы, которые “не знают, к какому классу относятся”.
Таким образом, статические методы прикреплены к классу лишь для удобства и не могут менять состояние ни класса, ни его экземпляра.

In [24]:
from datetime import date
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
    @classmethod
    def from_birth_year(cls, name, year):
        return cls(name, date.today().year - year)
    @staticmethod
    def is_adult(age):
        return age > 18


person1 = Person('Sarah', 25)
person2 = Person.from_birth_year('Roark', 1994)

person1.name, person1.age
#Sarah 25
person2.name, person2.age
#Roark 24
Person.is_adult(25)
#True

True

Чаще всего метод класса используется тогда, когда нужен генерирующий метод, возвращающий объект класса. Как видим, метод класса from_birth_year используется для создания объекта класса Person по году рождения, а не возрасту.
Статические методы в основном используются как вспомогательные функции и работают с данными, которые им передаются.

Методы экземпляра класса получают доступ к объекту класса через параметр self и к классу через `self.__class__`.
Методы класса не могут получить доступ к определенному объекту класса, но имеют доступ к самому классу через cls.
Статические методы работают как обычные функции, но принадлежат области имен класса. Они не имеют доступа ни к самому классу, ни к его экземплярам.

# Свойства, декоратор @property

In [25]:
class Person:
    def __init__(self, name, surname):
        self.name = name
        self.surname = surname

In [26]:
class Person:
    def __init__(self, name, surname):
        self.name = name
        self.surname = surname
        
    @property
    def full_name(self):
        return self.name + ' ' + self.surname

tom = Person('Thomas', 'Smith')
tom.full_name
# 'Thomas Smith'

'Thomas Smith'

In [27]:
tom.name = 'Alice'
tom.full_name
# 'Alice Smith'

'Alice Smith'

In [28]:
class Person:
    def __init__(self, name, surname):
        self.name = name
        self.surname = surname

    @property
    def full_name(self):
        return self.name + ' ' + self.surname

    # сеттер для свойства full_name
    @full_name.setter
    def full_name(self, new):
        self.name, self.surname = new.split(' ')

In [29]:
tom = Person('Thomas', 'Smith')
tom.full_name
# 'Thomas Smith'

tom.full_name = 'Alice Cooper'
tom.name
# 'Alice'

tom.surname
# 'Cooper'

'Cooper'

```python
property(fget=None, fset=None, fdel=None, doc=None)

def get_full_name(self):
    ...

def set_full_name(self, new):
    ...

class Person:
    ...
    full_name = property(
        fget=get_full_name,
        fset=set_full_name,
        doc='A full name of person'
    )
```

# Наследование

In [30]:
class Parent:
    def __init__(self):
        self.value = 10
    
    def get_value(self):
        return self.value

class Child(Parent):
    pass

class NewChild(Parent):
    def __init__(self):
        super().__init__()  # without it we will not have `value` field
        self.new_value = 20
    
print(Parent().get_value(), Child().get_value(), NewChild().get_value())

print(Child().__dict__)
print(NewChild().__dict__)

10 10 10
{'value': 10}
{'value': 10, 'new_value': 20}


#Перегрузка родительских методов

In [31]:
class Parent:
    def __init__(self, value):
        self._value = value
    
    def get_value(self):
        return self._value

    def __str__(self):
        return f'Value: {self._value}'

class Child(Parent):
    def __init__(self, value):
        Parent.__init__(self, value)  # == super().__init__(value)

    def get_value(self):
        return Parent.get_value(self) * 2  # == super().get_value() * 2

print(Parent(10).get_value())
print(Child(10).get_value())
print(Child(10)._value)
print(Child(10))

10
20
10
Value: 10


# Искаженные (mangled) атрибуты

Главное название такого именования - борьба с коллизией важных полей при наследовании (аналог `final` атрибутов в Java):

In [32]:
class Parent:
    def __init__(self):
        self.__variable = 'Parent'

class Child(Parent):
    def __init__(self):
        self.__variable = 'Child'

print(Parent().__dict__)
print(Child().__dict__)

# Parent.__variable
Parent()._Parent__variable

{'_Parent__variable': 'Parent'}
{'_Child__variable': 'Child'}


'Parent'

# Множественное наследование

- Даёт возможность классу получить методы и свойства сразу нескольких предков<br><br>
- Позволяет строить сложные иерархии зависимостей<br><br>
- Использовать без необходимости не нужно, поскольку архитектура кода сильно усложняется<br><br>
- В Python поддерживается без ограничений<br><br>
- У класса может быть один и более предков (`object` есть всегда)

# Множественное наследование в Python
- Методы и атрибуты ищутся в следующем порядке:<br><br>
    1. имя ищется в объекте (т.е. в его `__dict__`)
    2. дальше в классе объекта
    3. дальше в предках класса<br><br>
- Для поиска по предкам используется MRO (Method resolution order)<br><br>
- У каждого класса в момент создания вычисляется этот порядок и сохраняется в атрибуте `__mro__`

In [33]:
class A():
    def method(self): return 'A'

class B():
    def method(self): return 'B'

class C(A, B): pass

print(C.__mro__)
print(C().method())

(<class '__main__.C'>, <class '__main__.A'>, <class '__main__.B'>, <class 'object'>)
A


# Вызов метода класса-родителя

- В Python есть два способа обращения к родительским методам<br><br>
    - через функцию `super` (использует порядок mro)
    - напрямую по имени<br><br>

In [34]:
class A():
    def method(self): return 'A'

class B():
    def method(self): return 'B'

class C(A, B):
    def method(self):
        return (A.method(self), B.method(self), super().method())

print(C.__mro__)
print(C().method())

(<class '__main__.C'>, <class '__main__.A'>, <class '__main__.B'>, <class 'object'>)
('A', 'B', 'A')


# Декораторы классов

### `Пример декоратора функции`

Опишем декоратор, замеряющий время работы функции:

In [35]:
def timed(callable_obj):
    import time

    def __timed(*args, **kwargs):
        time_start = time.time()
        result = callable_obj(*args, **kwargs)
        time_end = time.time()
        
        print('{}  {:.3f} ms'.format(callable_obj.__name__,
                                     (time_end - time_start) * 1000))
        return result

    return __timed

In [36]:
@timed
def func():
    for i in range(1000000):
        pass
func()

func  29.202 ms


### `Пример декоратора класса`

Опишем декоратор класса, который получает на вход декоратор функции и оборачивает его вокруг каждого публичного метода класса:

In [37]:
def decorate_class(decorator):
    def _decorate(cls):
        for f in cls.__dict__:
            if callable(getattr(cls, f)) and not f.startswith("_"):
                setattr(cls, f, decorator(getattr(cls, f)))
        return cls
    return _decorate

In [38]:
@decorate_class(timed)
class Cls:
    a = 10
    def method(self): pass
    def _method_2(self): pass

Cls().method()
Cls().a  # not callable
Cls()._method_2()  # not public

method  0.001 ms


При определении класса производится вызов `decorate_class._decorate`, которая возвращает обновлённый класс.

# Магические методы

### `Class magic methods`

- Магические методы придают объекту класса определенные свойства

- Такие методы вызываются интерпретатором неявно

- Например, операторы - это магические методы<bf>

Рассмотрим несколько примеров:

In [39]:
class Cls:
    def __init__(self):  # initialize object
        self.name = 'Some class'
    
    def __repr__(self):  # str for printing object
        return 'Class: {}'.format(self.name)
    
    def __call__(self, counter):  # call == operator() in C++
        return self.name * counter

cls = Cls()
print(cls.__repr__())  # == print(cls)
print(cls(2))

Class: Some class
Some classSome class


### `Class magic methods`

Еще примеры магических методов:

In [40]:
def __lt__(self, other): pass

def __eq__(self, other): pass

def __add__(self, other): pass

def __mul__(self, value): pass

def __int__(self): pass

def __bool__(self): pass

def __hash__(self): pass

def __getitem__(self, index): pass

def __setitem__(self, index, value): pass

### `Как на самом деле устроен доступ к атрибутам`

При работе с атрибутами вызываются магические методы `__getattr__`, `__getattribute__`, `__setattr__` и `__delattr__`:

In [41]:
class Cls:
    def __setattr__(self, attr, value):
        print(f'Create attr with name "{attr}" and value "{value}"')
        self.__dict__[attr] = value

    def __getattr__(self, attr):
        print(f'WE WILL ENTER IT ONLY IN CASE OF ERROR!')
        return self.__dict__[attr]

    def __getattribute__(self, attr):
        if not attr.startswith('__'):
            print(f'Get value of attr with name "{attr}"')
            
        return super().__getattribute__(attr)  # call parent method implementation
        
    def __delattr__(self, attr):
        print(f'Remove attr "{attr}" is impossible!')
    

### `Как на самом деле устроен доступ к атрибутам`

In [42]:
cls = Cls()

cls.some_attr = 'some'
a = cls.some_attr

del cls.some_attr
b = cls.some_attr
cls.non_exists_attr

Create attr with name "some_attr" and value "some"
Get value of attr with name "some_attr"
Remove attr "some_attr" is impossible!
Get value of attr with name "some_attr"
Get value of attr with name "non_exists_attr"
WE WILL ENTER IT ONLY IN CASE OF ERROR!


KeyError: ignored

### `Магические методы и менеджер контекста`

Менеджер контекста (оператор `with`) работает с двумя магическими методами:

- `__enter__` - код, который нужно выполнить над объектом при входе в блок менеджера
- `__exit__` - код, который нужно в любом случае выполнить при выходе из блока

In [43]:
class SomeDataBaseDao:
    def __init__(self): self._db = ()
    
    def append(self, value): self._db.append(value)
    
    def __enter__(self):
        self._db = list(self._db)
        print('Set DB to read-write mode')
        return self

    def __exit__(self, exception_type, exception_val, trace):
        self._db = tuple(self._db)
        print('Set DB to read-only mode')
        return True

dao = SomeDataBaseDao()
#dao.append(1)  # AttributeError: 'tuple' object has no attribute 'append'
with dao:
    dao.append(1)
print(dao._db)

Set DB to read-write mode
Set DB to read-only mode
(1,)
